In [20]:
import os
import pandas as pd

csv_dir = '/home/ksaff/Desktop/ttyd/fine_tuning/llama_finetuning/10_datasets'
ddl_dir = '/home/ksaff/Desktop/ttyd/fine_tuning/fine_tune_dbs/ddls'

csv_names = sorted(os.listdir(csv_dir))[:10]
ddl_names = sorted(os.listdir(ddl_dir))
ddl_names.pop(2) #Del DDL10

fine_tune_0.csv ddl0.sql
fine_tune_1.csv ddl1.sql
fine_tune_2.csv ddl2.sql
fine_tune_3.csv ddl3.sql
fine_tune_4.csv ddl4.sql
fine_tune_5.csv ddl5.sql
fine_tune_6.csv ddl6.sql
fine_tune_7.csv ddl7.sql
fine_tune_8.csv ddl8.sql
fine_tune_9.csv ddl9.sql


In [23]:
import re

def rewrite_DDL(DDL):
    lines = DDL.split('\n')
    modified_text = "DDL: "
    for line in lines:
        if len(line) < 2:
            pass
        elif '/*' in line:
            pass
        elif 'DROP TABLE' in line:
            pass

        else:
            line = re.sub(r'(?<!NOT )\bNULL\b', '', line)
            modified_text += f'{line}\n'
            
    return modified_text

dfs = []

for csv_file, ddl_file in zip(csv_names, ddl_names):
    csv_path = os.path.join(csv_dir, csv_file)
    ddl_path = os.path.join(ddl_dir, ddl_file)

    df = pd.read_csv(csv_path)
    with open(ddl_path) as file:
        ddl = file.read()
    ddl = rewrite_DDL(ddl)
    df = df.T.reset_index().T
    new_columns = ['instruction', 'output']
    df.columns = new_columns
    df.index = range(0, len(df))
    df['input'] = ddl
    dfs.append(df)

In [36]:
df = pd.concat(dfs, ignore_index=True)

In [40]:
text_col = []

for _, row in df.iterrows():
    prompt = 'Make SQLite query based on DDL and instruction.'
    instruction = row['instruction']
    input_query = row['input']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Input:\n'
        + input_query
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

In [41]:
df.to_csv('train.csv', index=False)
# df[20:].to_csv('train.csv', index=False)
# df[:20].to_csv('test.csv', index=False)
